In [1]:
import os
from itertools import product
import re
from fit import correlation_matrix
from fit import fit_line
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.pipeline import Pipeline
import django
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.cm import register_cmap
from matplotlib.colors import ListedColormap
from sklearn.decomposition import PCA, FactorAnalysis
import marslab.spectops as ops
from fit import correlation_matrix
from marslab.imgops.imgutils import normalize_range
from marslab.compat.xcam import DERIVED_CAM_DICT
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.pipeline import Pipeline
from marslab.imgops.pltutils import attach_axis, set_colorbar_font


from correlate import (
    s_from_midnight, explode_binary, translate_fields_for_corr_graphs,
    preprocess_for_corrs, plot_dimensionality_matrices,
    plot_mdex_pca
)

# os.chdir("..")

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "multidex.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()
import plotter.models
from plotter.spectrum_ops import filter_df_from_queryset
from multidex_utils import model_metadata_df

%matplotlib qt
%matplotlib qt

ImportError: cannot import name 'preprocess_for_corrs' from 'correlate' (/home/michael/Desktop/multidex_future/multidex/scratch/correlate.py)

In [ ]:
def explained_variance_ratios(array):
    variances = array.var(axis=0)
    total = sum(variances)
    return variances / total

In [ ]:
instrument = "ZCAM"
filter_info = DERIVED_CAM_DICT[instrument]["filters"]
filters = list(filter_info.keys())
narrowband = [filt for filt in filters if not re.match(r"[LR]0[RGB]", filt)]


In [ ]:
plt.rcParams["figure.figsize"] = (11, 11)
corrs = []
transforms = []
figlist = []
for r_star, scale_to, search_terms, norm_values, filts in product(
    [False, True],
#     [None, ("L1", "R1")],
    [None],
    [None],
#     [None, [("feature", "rock")]],
    [False, 'R6'],
#     [PCA, FactorAnalysis]
    [filters, narrowband]
    
):
#     if search_terms is not None:
#         explode_field = "morphology"
#     else:
#         explode_field = None
    if (scale_to is not False) and (r_star is True):
        continue     # pointless, move on
    explode_field='None'
    method=PCA
    correlations, transform, figs = plot_mdex_pca(
        scale_to = scale_to,
        instrument=instrument,
        explode_field = explode_field,
        r_star = r_star,
        search_terms = search_terms,
        corr_fields = filts + [
            'incidence_angle'
        ],
        pca_fields=filts,
        corr_cmap="orange_teal",
        fontsize=18,
        method=method,
        norm_values=norm_values
        
    )
    corrs.append(correlations)
    transforms.append(transform)
    figlist.append(figs)

In [ ]:
plt.close('all')

In [ ]:
for figs in figlist:
    figs['pc correlations'].show()

In [ ]:
correlation_matrix(corpus[['zoom', 'sol']])

In [ ]:
corpus['zoom']

In [ ]:
narrowband = [filt for filt in filters if not re.match(r"[LR]0[RGB]", filt)]
search_terms=None
norm_values = False
pca_fields = filters
corr_fields = filters + ["incidence_angle", 'r16', 'r56', 'l645']

# corr_fields = filters + ['incidence_angle']

corr_fields = filters 
explode_field = 'feature'

# scale_to=('L1', 'R1')

# corpus["ltst"] = corpus["ltst"].map(s_from_midnight)
corpus['r56'] = corpus['R5'] / corpus['R6']
corpus['r16'] = corpus['R1'] / corpus['R6']
corpus['l645'] = ops.band_depth(
    corpus[['L6', 'L4', 'L5']].T,
    None,
    wave_mapping[['L6', 'L4', 'L5']]
)[0]


if 'zoom' in corpus.columns:
    corpus['zoom'] = corpus['zoom'].astype('float16')

# search = search.loc[search['feature'] == 'soil']
# fields to do pca on

pca_fields = translate_fields_for_corr_graphs(filters, pca_fields)

# fields to compare with the PCs
corr_fields = translate_fields_for_corr_graphs(filters, corr_fields)
if exploded is not None:
    corr_fields += list(exploded.columns)

# corr_fields += [band + "_err" for band in filters]
pca_data, corr_data = preprocess_for_corrs(
    corr_fields, norm_values, pca_fields, search, None
)
pca_data = search[pca_fields].dropna(axis=0)
corr_data = search[corr_fields].dropna(axis=0)

In [ ]:
corpu

In [ ]:
instrument = 'ZCAM'
spec_model = plotter.models.INSTRUMENT_MODEL_MAPPING[instrument]
wave_mapping = pd.Series(DERIVED_CAM_DICT[instrument]['filters'])
metadata_df = model_metadata_df(spec_model)
filter_info = DERIVED_CAM_DICT[instrument]["filters"]
filters = list(filter_info.keys())

In [ ]:
r_star=False
scale_to=None
# scale_to=('L6', 'R6')
explode_field='feature'
data_df = filter_df_from_queryset(
    spec_model.objects.all(), r_star=r_star, scale_to=scale_to
)
corpus = pd.concat([metadata_df, data_df], axis=1)

if (explode_field is not None) and (explode_field in corpus.columns):
    exploded = explode_binary(corpus, explode_field)
    search = pd.concat([corpus.copy(), exploded], axis=1)
else:
    exploded = None
    search = corpus.copy()
corr_fields = filters + list(exploded.columns)

search = corpus.copy()
search = search.dropna(
    subset=filters+['incidence_angle'], axis=0
)
search["avg"] = search[filters].mean(axis=1)

# search = search.loc[search['incidence_angle'] < 40].copy()
# search = search.loc[search['avg'] < 0.3].copy()
corr_data = search[corr_fields].dropna(axis=1)
correlations = correlation_matrix(corr_data)

avg = search['avg']
theta = search['incidence_angle']

In [ ]:
exploded

In [ ]:
# average of err across all filters as a parameter

In [ ]:
pipe = Pipeline(steps=[
    ('norm', Normalizer()),
    ('scale', StandardScaler()),
    ('reduce', PCA(n_components=8))
])

In [ ]:
vectors = pca_data.T.to_dict("list")
vectarray = np.array(tuple(vectors.values()))
transform = pipe.fit_transform(vectarray)
transform = pd.DataFrame(transform)
transform.columns = ["P" + str(column + 1) for column in transform.columns]


In [ ]:
plot_dimensionality_matrices(
    correlations, 
    None,
#     transform, 
    corr_fields, 
    corr_cmap='orange_teal', 
    fontsize=20,
    which='features',
    title=" ".join(pipe.named_steps.keys())

)

In [ ]:
1/np.cos(np.radians(30))

In [ ]:
plt.figure()
plt.title('no R*')
fit_line(avg, theta)

In [ ]:
plt.figure()
plt.title('R*')
fit_line(avg/np.cos(np.radians(theta)), theta)

In [ ]:
explained_variance_ratios(transform)

In [ ]:
corr_data

In [ ]:
# pipe = noscalepipe
# vectors = pca_data.T.to_dict("list")
# vectarray = np.array(tuple(vectors.values()))
# transform = pipe.fit_transform(vectarray)
# transform = pd.DataFrame(transform)
# transform.columns = [
#     "P" + str(column + 1) for column in transform.columns
# ]
# explained_variance_ratios(transform)
# if "explained_variance_ratio_" in dir(pipe.named_steps['reduce']):
#     explained_variance = np.round(
#         pipe.named_steps['reduce'].explained_variance_ratio_ * 100, 2
#     )
# else:
#     explained_variance = None
# corr_frame = pd.concat([corr_data, transform], axis=1)
corr_frame = corr_data
correlations = correlation_matrix(corr_frame)



In [ ]:
from sklearn.decomposition import FastICA
vectors = pca_data.T.to_dict("list")
vectarray = np.array(tuple(vectors.values()))
pipe = Pipeline(steps=[
    ('norm', Normalizer()),
    ('scale', StandardScaler()),
    ('ica', FastICA(n_components=8, whiten=True))
])
transform = pipe.fit_transform(vectarray)
transform = pd.DataFrame(transform)
transform.columns = ["P" + str(column + 1) for column in transform.columns]

# corr_frame = pd.concat([corr_data, transform], axis=1)
correlations = correlation_matrix(corr_frame)

plot_dimensionality_matrices(
    correlations, 
    transform, 
    corr_fields, 
    corr_cmap='orange_teal', 
    fontsize=20,
    which='parameters',
    title=" ".join(pipe.named_steps.keys())

)

In [ ]:
from sklearn.decomposition import NMF
vectors = pca_data.T.to_dict("list")
vectarray = np.array(tuple(vectors.values()))
pipe = Pipeline(steps=[
    ('norm', Normalizer()),
#     ('scale', StandardScaler()),
    ('nmf', NMF(n_components=16, init='random', max_iter=5000))
])
transform = pipe.fit_transform(vectarray)
transform = pd.DataFrame(transform)
transform.columns = ["P" + str(column + 1) for column in transform.columns]

corr_frame = pd.concat([corr_data, transform], axis=1)
correlations = correlation_matrix(corr_frame)

plot_dimensionality_matrices(
    correlations, 
    transform, 
    corr_fields, 
    corr_cmap='orange_teal', 
    fontsize=20,
    which='parameters',
    title=" ".join(pipe.named_steps.keys())

)

In [ ]:
explained_variance_ratios(transform)

In [ ]:
from sklearn.decomposition import KernelPCA
vectors = pca_data.T.to_dict("list")
vectarray = np.array(tuple(vectors.values()))
pipe = Pipeline(steps=[
    ('norm', Normalizer()),
    ('scale', StandardScaler()),
    ('kernel', KernelPCA(n_components=16))
])
transform = pipe.fit_transform(vectarray)
transform = pd.DataFrame(transform)
transform.columns = ["P" + str(column + 1) for column in transform.columns]

corr_frame = pd.concat([corr_data, transform], axis=1)
correlations = correlation_matrix(corr_frame)

plot_dimensionality_matrices(
    correlations, 
    transform, 
    corr_fields, 
    corr_cmap='orange_teal', 
    fontsize=20,
    which='parameters',
    title=" ".join(pipe.named_steps.keys())

)

In [ ]:
np.array([1,1,2,3,3]).var(axis=0)

In [ ]:
from sklearn.metrics import explained_variance_score

In [ ]:
transform

In [ ]:
vectarray.shape

In [ ]:
from fit import coef_det

In [ ]:
plot_dimensionality_matrices?